#Milestone Project 1: Food Vision Big

see the annotated version of this notebook on GitHub

In [2]:
# !pip install tensorflow==2.5.0
import tensorflow as tf
tf.__version__

'2.12.0'

In [5]:
!pip install protobuf==3.20.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached protobuf-3.20.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


## Check our GPU

* Google colab offers free GPUs (thank you Google), however, not all of them are compatible with mixed precision training.
Google Colab offers:
* K80 (Not compatible)
* P100 (not compatible)
* Tesla T4 (compatible)

Knowing this, in order to use mixed precision training we need access to a Tesla T4 ( from within Google Colab) or if we're using our own hardware, our GPU needs a score of 7.0+ (see here: https://developer.nvidia.com/cuda-gpus#compute)

In [6]:
#if the following line doesn't output "Tesla T4", you can try getting access to 
#another GPU by going to Runtime -> Factory Reset Runtime -> "Yes" and then
#rerunning this cell
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-e672a513-07c7-d422-f71d-e78dda85c625)


##Get helper functions

In past modules, we've created a bunch of helper functions to do small tasks required for our notebooks.

Rather than rewrite all of these, we can import a script and load them in from there.

The script we've got available can be found on GitHub: https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [7]:
#download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-04-26 06:34:41--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-04-26 06:34:41 (72.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [8]:
#Import series of helper functions
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

## Use tensorflow dataset to download data

If you want to get an overview of TensorFlow Dataset (TFDS), read the guide


In [9]:
#Get TensorFlow Dataset
import tensorflow_datasets as tfds


In [10]:
#list all available datasets
datasets_list = tfds.list_builders() #get all available datasets in TFDS

print("food101" in datasets_list) # is our target dataset in the list of TFDS datasets

True


In [13]:
# Load in the data (takes 5-6 minutes in Google Colab)
(train_data, test_data), ds_info = tfds.load(name = "food101",
                                             split=["train", "validation"],
                                             shuffle_files = True,
                                             as_supervised = True, #data returned in turple format (data, label)
                                             with_info = True)

ImportError: ignored

## Exploring the Food101 data from TensorFlow Dataset
To become one with our data, we want to find:
* Class names
* The shape of our input data (image tensors)
* The datatype of our input data
* What the labels look like (e.g. are they one-hot encoded or are they label encoded?)
* Do the label match up with the calss names?

In [12]:
#Features of Food101 from TFDS
ds_info.features

NameError: ignored

In [ ]:
#Get the class names
class_names = ds_info.features["label"].names
class_names[:10]

In [ ]:
#Take one sample of the train data
train_one_sample = train_data.take(1) # sample are in format (image_tensor, label)

In [ ]:
#what does one sample of our training data look like
train_one_sample

In [ ]:
#output info about our training sample
for image, label in train_one_sample:
  print(f"""
  Image shape: {image.shape},
  Image datatype: {image.dtype},
  Target class from Food101 (tensor form): {label},
  Class name (str form): {class_names[label.numpy()]}
  
  """)

In [ ]:
#What does our image tensor from TFDS's Food101 look like?
image

In [ ]:
import tensorflow as tf
#what are the min and max values of our image tensor
tf.reduce_min(image), tf.reduce_max(image)

### Plot an image from tensorFlow Datasets


In [ ]:
ds_info

In [ ]:
import matplotlib.pyplot as plt


#Plot an image tensor
target_data = train_data.take(1)
plt.figure()
for image, label in target_data:
  plt.imshow(image)
  plt.title(f"label:{class_names[label.numpy()]}") #add title to image to verify the label is associate with the right image
  plt.axis(False)

##Create preprocessing functions for our data

Neural networks perform best when data is in a certain way (e.g. batched, normalized, etc)

However, not all data (including data from TensorFlow Datasets) comes like this

So in order to get it ready for a neural network, you 'll often have to write preprocessing functions and map it to your data.

What we know about our data:
* In `uint8` datatype
* Comprised of all different size tensors (different sized images)
* Not scaled (the pixel values are between 0 and 255)

What we know models like:
* Data in `float32` dtype (or for mixed precision `fload16` and `float32`)
* For batches, TensorFlow likes all of the tensors within a batch to be the same size
* Scaled (values between 0 & 1) also called normalized tensors generally perform better

With these points in mind, we've got a few things we can tackle with a preprocessing function.

Since we're going to be using an EfficientNetBX pretrained model from tf.keras.applications, we don't need to rescale our data (these architecture have rescaling built-in)

This means our functions need to:
1. Reshape our images to all the same size
2. Convert the dtype of our image tensors from `uint8` to `float32`

In [ ]:
#make a function for preprocessing images
def preprocess_img(image, label, image_shape = 224):
  """
  Converts image data type from 'uint8' -> 'float32'
  Reshapes image to (image_shape, image_shape, color_channels)
  """
  image = tf.image.resize(image, [image_shape, image_shape]) #reshape target image
  # image = image/255. #scale image values => not required with EfficientNetBX model from tf.keras.applications
  return tf.cast(image, dtype = tf.float32), label # return (float32_image, label) tuple


In [ ]:
#Preprocess a single sample image and check the outputs
preprocessed_image = preprocess_img(image, label)[0]
print(f"image before preprocessing:\n {image[:2]}...., \nShape: {image.shape}, \nDatatype: {image.dtype}\n")
print(f"image after preprocessing:\n {preprocessed_image[:2]},...., \nShape: {preprocessed_image.shape}, \nDatatype: {preprocessed_image.dtype}\n" )

##Batch and prepare datasets

We're now going to make our data input pipeline run really fast.

For more resources on this, I'd highly going through the following guide: https://www.tensorflow.org/guide/data_performance


In [ ]:
#map preprocessing function to training (and parallelize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls = tf.data.AUTOTUNE)

#Shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size = 32).prefetch(buffer_size=tf.data.AUTOTUNE)

#map preprocessing function to test data
test_data = test_data.map(map_func = preprocess_img, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(buffer_size = tf.data.AUTOTUNE)

#

In [ ]:
train_data, test_data

> "Hey, Tensorflow, map this preprocessing function(`preprocess_img`) accross our training dataset, then shuffle a number of elements and then batch them together and finally make sure you prepare new batches (prefetch) whilst the model is looking through (finding patterns) the current batch"

## Create modelling callback

We're going to create a couple of callbacls to help us while our model trains:
* TensorBoard callback to log training results ( so we can visualize them later if need to be)
* ModelCheckpoint callback to save our model's progress after feature extraction

In [ ]:
# Create tensorboard callback (inport from helper_function.py)

from helper_functions import create_tensorboard_callback

#create a model checkpoint callback to save our model's progress during training
checkpoint_path = "model_checkpoints/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor = "val_acc",
                                                      save_best_only = True,
                                                      save_weights_only=True,
                                                      verbose = 0)# don't print whether or not model is being saved

## Setup mixed precision training

First and foremost, for a deeper understanding of mixed precision training, check out the TensorFlow guide for mixed precision: https://www.tensorflow.org/guide/mixed_precision

Mixed precision utilizes a combination of float32 and float16  data types to speed up model performance

In [ ]:
# Turn on mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16") #set global data policy to mixed precision

In [ ]:
mixed_precision.global_policy()

## Build feature extraction model 


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Create base model
input_shape = (224,224,3)

base_model =tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False

# Create functional model
inputs = layers.Input(shape = input_shape, name="input_layer")
#note: efficientnetBX models have rescaling built-in but if your model doesn't you can have a layer like below
#x = preprocessing.Rescaling(1./255)(inputs)

x = base_model(inputs, training = False) # make sure layers which should be in inference must only stay the same
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype = tf.float32, name = "softmax_float32")(x)

model = tf.keras.Model(inputs, outputs)


#compile our model
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ["accuracy"])


In [ ]:
model.summary()

##Checking layer dtype policies (are we using mixed precision?)


In [ ]:
#Check the dtype_policy attributes of layers in our model
for layer in model.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

Going through the above we see:
* `layer.name`: the human readable name of a particular layer
* `layer.trainable` : is the layer trainable or not (if `False`, the weights are frozen)
* `layer.dtype`: the data type a layer stores its variables in 
* `layer.dtype_policy`: the data type policy a layer computes on its variables with  

In [ ]:
# Check the dtype_policy attributes of layers in base model
for layer in model.layers[1].layers[:20]: #check the layers of the base model(layer at the index 1 of model)
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

##Fit the feature extraction model

if our goal is to fine-tune a pretrained model, the general order of doing thing is:
  1. build a feature extraction model (train a couple output layers with base layers frozen)
  2. Fine-tune some of the frozen layers

In [ ]:
#Fit the model
model_101_food_classes_feature_extract = model.fit(train_data,
                                                   epochs = 3,
                                                   steps_per_epoch = len(train_data),
                                                   validation_data = test_data,
                                                   validation_steps = int(0.15*len(test_data)),
                                                   callbacks = [create_tensorboard_callback("training_logs", "efficientnetb0_101_classes_all_data_feature_extract"), 
                                                                model_checkpoint])

In [ ]:
#Evaluate model on whole test dataset
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model

#Off to you...

We've covered alot of ground so far but since this is a milestone project, it's time for you to takeover.

More specifically, you're challenge is to complete the TEMPLATE version of 07 to obtain a computer vision model (building off the one we've built in this notebook) to beat the DeepFood paper (77,4%)

In other words, you're going to create food vision big